## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_vacation import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jiexiu,CN,37.0244,111.9125,67.21,37,9,2.08,clear sky
1,1,Castro,BR,-24.7911,-50.0119,67.69,76,100,7.90,overcast clouds
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,34.25,74,8,11.25,clear sky
3,3,Esperance,AU,-33.8667,121.9000,56.08,61,100,3.71,overcast clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,38.82,75,20,5.75,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Hilo,US,89.28,clear sky,19.7297,-155.0900,
11,Anito,PH,76.84,light rain,12.4497,125.2886,
14,Atuona,PF,78.03,few clouds,-9.8000,-139.0333,
23,Albany,US,80.64,overcast clouds,42.6001,-73.9662,
26,Butaritari,KI,80.55,light rain,3.0707,172.7902,
28,Inongo,CD,76.17,overcast clouds,-1.9500,18.2667,
33,Itarema,BR,82.38,few clouds,-2.9248,-39.9167,
35,Machilipatnam,IN,87.37,overcast clouds,16.1667,81.1333,
38,Jamestown,US,80.02,overcast clouds,42.0970,-79.2353,
41,Hithadhoo,MV,82.27,overcast clouds,-0.6000,73.0833,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
clean_hotel_df = clean_hotel_df.replace(r'^s*$', float('NaN'), regex = True)
clean_hotel_df.dropna(inplace = True)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Hilo,US,89.28,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
11,Anito,PH,76.84,light rain,12.4497,125.2886,El Felix Cottages
14,Atuona,PF,78.03,few clouds,-9.8000,-139.0333,Villa Enata
26,Butaritari,KI,80.55,light rain,3.0707,172.7902,Isles Sunset Lodge
28,Inongo,CD,76.17,overcast clouds,-1.9500,18.2667,Hotel Du Lac
33,Itarema,BR,82.38,few clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
35,Machilipatnam,IN,87.37,overcast clouds,16.1667,81.1333,R&B Bungalow
38,Jamestown,US,80.02,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
41,Hithadhoo,MV,82.27,overcast clouds,-0.6000,73.0833,Scoop Guest House
43,Lorengau,PG,80.51,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]

max_temp = clean_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))